## Notebook for reconstructing from a single alinged tilt-series

### 1. 3-d mask adjustments
### 2. Reconstruction
### 3. Diagnostics

In [1]:
%matplotlib qt

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pickle
from timeit import default_timer as timer
import skimage.filters as skfl

In [3]:
import hyperspy.api as hs

In [4]:
#load editable pyramid version
import sys
import os
pyramid_dev_version_path=os.path.abspath(r"D:\Programmes\Git\empyre-patched\empyre")
sys.path.append(pyramid_dev_version_path)
#print(sys.path)
import pyramid as pr
pr

<module 'pyramid' from 'D:\\Programmes\\Git\\empyre-patched\\empyre\\pyramid\\__init__.py'>

In [5]:
#Code by A.S.
import pyramidas.alignment as pa
import pyramidas.util as pu
import pyramidas.reconstruction as pre
import pyramidas.diagnostics as pd

### 2. Reconstruction

In [6]:
with open("data_combined_refined.pickle", "rb") as f: #data_combined_refined
    data_combined=pickle.load(f)
    mask_edge=5

In [57]:
data_combined.plot_mask()

In [58]:
result_folder=".//"
results={}

In [65]:
### Try different regulariser strengths
data=data_combined
regulariser_strengths = 1e1
mag_guess = None
 #x-axis edge of boundary charges
regulariser_mask=data.mask.copy()
regulariser_mask[:,:,:mask_edge]=False
iterations=6000
sinle_run=2000 #iterations
result_note=f"{iterations} iterations"
cost_values=[]

#because Jutil solver accumulates errors, restart the reconstruction regularly be supplying the input
start_time = timer()
runs=iterations//sinle_run
magdata_rec=mag_guess
for i in range(runs):

    #impose uniformity after obtaining rough starting solution
    magdata_rec, cost_function = pre.reconstruct_from_phasemaps(data, lam=regulariser_strengths, 
                                        max_iter=sinle_run, ramp_order=1, verbose=False, plot_input=False, plot_results=False, 
                                       regulariser_type='exchange',mag_0=magdata_rec, reg_mask=regulariser_mask)
    pre.append_valid_costfunction_values(cost_values, cost_function)
    print(f"Finished run {i+1}/{runs}, with cost {cost_function.chisq}")
    
end_time = timer()
execution_time = end_time - start_time #seconds
print(f"calculating {iterations} iterations took {execution_time/60:.1f} minutes")

    
result = (magdata_rec, cost_function, cost_values, result_note)
results[regulariser_strengths] = result

Finished run 1/3, with cost 29124.926774452753
Finished run 2/3, with cost 29124.927803458933
Finished run 3/3, with cost 29124.927612966425
calculating 6000 iterations took 44.2 minutes


In [66]:
angle_field = pre.inspect_magdata(magdata_rec[:,:,:,mask_edge:], plot_angles=True)
pre.inspect_cost_values(cost_values, print_chis=True)

Max spin angle: 164.45569550203405

N = 3 cost value pairs.
model, regulariser, sum
2.69342e+04; 2.19068e+03; 2.91249e+04
2.69343e+04; 2.19068e+03; 2.91249e+04
2.69343e+04; 2.19068e+03; 2.91249e+04



In [67]:
plt.matshow(np.max(angle_field.field,axis=0))

In [ ]:
t=magdata_rec[...,mask_edge:].plot_quiver3d(mode='arrow', ar_dens=2)

In [ ]:
t=pre.simulate_reconstruction(data, magdata_rec, cost_function)

In [69]:
with open(result_folder+"reconstruction_results_e1_10000_mask_corrected.pickle","wb") as f:
    pickle.dump(results, f)

### 3.Diagnostics

In [63]:
import matplotlib as mpl
mpl.rcParams.update({"figure.dpi":200.0, 
                     'figure.figsize': [6.0, 4.0],
                     'font.size': 12.0, 
                     'legend.fontsize': 12.0,
                     'lines.linewidth': 1})

In [70]:
with open("data_combined_refined.pickle", "rb") as f: #data_combined_refined
    data_refined=pickle.load(f)
    mask0 = data_refined.mask.copy()
    
result_folder=".//"
with open(result_folder+"reconstruction_results_e1_10000_mask_corrected.pickle", "rb") as f:
    results=pickle.load(f)
key=(1e1)
magdata_rec, cost_function, cost_values, result_note = results[key]
mask_edge=5
print("Weights:",key,"Note:",result_note)

#select the voxel to evaluate
voxel_position = pd.find_voxel_coords(data_refined.mask, position_zyx=[6,40,50], mask_edge=mask_edge, plot_results=True)

#Calculate information content distributions
magdata_rec, cost_function, cost_values = results[key][:3]
mag_rec = magdata_rec
cost_f = cost_function
data = data_refined

diagnostic_results = pd.bayesian_diagnostics(data, mag_rec, cost_f, voxel_position_zyx=voxel_position, 
                       verbose=True, max_iter=300, plot_results=True)

print("spatial resolution FWHM")
print(np.sqrt(np.average(diagnostic_results[1]**2))*10.25)

# with open(result_folder+"diagnostics_em1e0_10000_mask_corrected.pickle","wb") as f:
#     pickle.dump(diagnostic_results, f)

Weights: 10.0 Note: 6000 iterations
finished calculating for x component
finished calculating for y component
finished calculating for z component

Voxel position: (6, 40, 50)
Magnetisation vector is:
        (M_x = 5.50e-01 +/- 2.74e-02 T,
         M_y = 3.10e-01 +/- 2.07e-02 T,
         M_z = 8.62e-02 +/- 5.65e-03 T)
Amplitude: 0.637 +/- 0.035 T
Spatial resolution (dx, dy, dz): 6.0, 4.4, 8.8 pixels
Pixel spacing: 10.25 nm
spatial resolution FWHM
67.98533530109366


In [71]:
with open(result_folder+"diagnostics_e1_10000_mask_corrected.pickle","wb") as f:
    pickle.dump(diagnostic_results, f)

In [45]:
np.sqrt(np.average(diagnostic_results[1]**2))*10.25

43.17481688250144

In [ ]:
angle_field = pre.inspect_magdata(magdata_rec[...,mask_edge:], plot_angles=True, ar_dens=2)
pre.inspect_cost_values(cost_values, print_chis=True)

In [ ]:
#select the voxel to evaluate
voxel_position = pd.find_voxel_coords(data_refined.mask, position_zyx=[5,40,50], mask_edge=mask_edge, plot_results=True)

In [ ]:
#Calculate information content distributions
magdata_rec, cost_function, cost_values = results[key][:3]
mag_rec = magdata_rec
cost_f = cost_function
data = data_refined

diagnostic_results = pd.bayesian_diagnostics(data, mag_rec, cost_f, voxel_position_zyx=voxel_position, 
                       verbose=True, max_iter=300, plot_results=True)

In [ ]:
print("spatial resolution FWHM")
np.sqrt(np.average(diagnostic_results[1]**2))*10.25

In [ ]:
# with open(result_folder+"diagnostics_e0.pickle","wb") as f:
#     pickle.dump(diagnostic_results, f)

In [ ]:
# with open(result_folder+"diagnostics_e0.pickle", "rb") as f:
#     diagnostic_results=pickle.load(f)

In [ ]:
pd.plot_error_gain_maps(data_refined, diagnostic_results, component='x')

### Histogram

In [9]:
#histograms
pd.histogram_magnetisation(magdata_rec[:,:,:,mask_edge:], [0,2], fit_gauss=False)

bin_size: 0.02 T
mean and std: 0.8531949149935405 0.37593808850896704


(array([  0.,   0.,   1.,   0.,   2.,   2.,   7.,  10.,  22.,  18.,  26.,
         37.,  53.,  54.,  75.,  89., 107., 105., 115., 151., 146., 180.,
        197., 245., 233., 248., 255., 239., 201., 207., 199., 222., 204.,
        202., 229., 221., 206., 188., 217., 223., 204., 207., 225., 199.,
        193., 194., 176., 177., 175., 180., 215., 190., 185., 148., 166.,
        129., 157., 157., 104., 122., 112., 107., 113.,  86.,  80., 101.,
         80.,  97.,  82.,  67.,  66.,  59.,  50.,  51.,  46.,  40.,  40.,
         30.,  23.,  23.,  32.,  29.,  26.,  24.,  17.,  18.,  19.,  21.,
          9.,  19.,  11.,  17.,   7.,  18.,   9.,  11.,  15.,  19.,   8.,
          8.]), 0.02)

### FSC

In [10]:
#FSC
#do for all sub-components:
mag_rec = magdata_rec[:,:,:,mask_edge:]

field_array=mag_rec.field.copy()
field_array[:,:,:,:mask_edge]=0

FSCs = []
resolutions=[]
for i in range(3):
    array_3d=field_array[i,:,:,:]
    ffts = pd.fsc_split_array(array_3d)
    freq, FSC, ns_effective = pd.fsc_calculate_correlation(array_3d, *ffts, scale=3, plot_results=True)
    FSCs.append(FSC)

C:\Users\LOCAL_Aurys\HyperSpy-bundle\envs\pyramidPatched2\lib\site-packages\skimage\transform\_warps.py:23: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '
C:\Users\LOCAL_Aurys\HyperSpy-bundle\envs\pyramidPatched2\lib\site-packages\skimage\transform\_warps.py:23: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '
C:\Users\LOCAL_Aurys\HyperSpy-bundle\envs\pyramidPatched2\lib\site-packages\skimage\transform\_warps.py:23: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting w

In [13]:
sne=np.sqrt(ns_effective)
sigma_3 = 3/(sne+2)
hbit=((0.2071+1.9102/sne)/(1.2071+0.9102/sne))

freq_p=2*np.array(freq) #for plotting convert to f/nyquist

def find_intersection(FSC, hbit, freq_p):
    for i in range(len(FSC))[10:]:
        fi=FSC[i]
        thi=[hbit[i]]
        if thi>fi:
            resolution = tick_function([freq_p[i-1]])[0]
            return(resolution)
            break
            
def tick_function(f):
    freqs=[]
    for freq in f:
        if freq<=1e-10:
            freqs.append(None)
        else:
            f=1/freq*mag_rec.a
            freqs.append("%.1f"%f)
    return (freqs) #in nm

plt.figure()
ax1 = plt.gca()
ax2 = ax1.twiny()

s='xyz'
fmts=["r.-","g.-","b.-"]
for i,FSC in enumerate(FSCs):
    resolution = find_intersection(FSC, hbit, freq_p)
    
    ax1.plot(freq_p, FSC, fmts[i], label=f"$m_{s[i]}$, resolution $= {resolution}$ nm")

ax1.plot(freq_p, hbit, 'k-', label="1/2 bit threshold")
#ax1.plot(freq_p, sigma_3, 'k-.', label="3 sigma")
ax1.set_xlabel("Spatial frequency / Nyquist")
ax1.legend(loc='lower left')
ax1.set_ylabel("Correlation coefficient")


xtickslocs = ax1.get_xticks()[1:-1]
ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(xtickslocs)
ax2.set_xticklabels(tick_function(xtickslocs))
ax2.set_xlabel("Resolution, nm")
plt.tight_layout()
plt.savefig("FSC.png",dpi=200)


### plotting

In [28]:
#for plotting 
with open("data_combined_refined.pickle", "rb") as f:
    data_refined=pickle.load(f)
    mask0 = data_refined.mask.copy()
    
result_folder=".//"
with open(result_folder+"reconstruction_results_em1e0_10000_mask_corrected.pickle", "rb") as f:
    results=pickle.load(f)
key=(1e-1,1e-0)
magdata_rec, cost_function, cost_values, result_note = results[key]
mask_edge=5
print("Weights:",key,"Note:",result_note)

Weights: (0.1, 1.0) Note: 10000 iterations


In [9]:
angle_field = pre.inspect_magdata(magdata_rec[:,:,:,mask_edge:], plot_angles=True)


Max spin angle: 174.71739245264737


In [10]:
pre.inspect_cost_values(cost_values, print_chis=True, scale='not log')


N = 5 cost value pairs.
model, regulariser, sum
2.48153e+04; 7.76239e+02; 2.55915e+04
2.47695e+04; 7.83207e+02; 2.55527e+04
2.47583e+04; 7.89113e+02; 2.55474e+04
2.47561e+04; 7.90496e+02; 2.55466e+04
2.47553e+04; 7.91000e+02; 2.55463e+04



In [39]:
pd.histogram_magnetisation(magdata_rec[...,mask_edge:],range_hist=(0,2) )

bin_size: 0.02 T
fitting params (a,b,c) with error: [236.03521899   0.80080298   0.36850077] [7.38042736 0.01329792 0.01362243]
mean and std: 0.8220901932384945 0.31551156867126356


(array([  0.,   0.,   1.,   0.,   2.,   2.,   7.,  10.,  22.,  18.,  26.,
         37.,  53.,  54.,  75.,  89., 107., 105., 115., 151., 146., 180.,
        197., 245., 233., 248., 255., 239., 201., 207., 199., 222., 204.,
        202., 229., 221., 206., 188., 217., 223., 204., 207., 225., 199.,
        193., 194., 176., 177., 175., 180., 274., 230., 223., 160., 190.,
        163., 181., 193., 162., 163., 154., 126., 113.,  95.,  85.,  97.,
         70.,  73.,  73.,  69., 218., 130.,  48.,  18.,  12.,   3.,   1.,
          0.,   2.,   1.,   1.,   1.,   1.,   0.,   0.,   0.,   0.,   1.,
          0.,   1.,   1.,   0.,   0.,   1.,   0.,   0.,   1.,   1.,   0.,
          0.]), 0.02)

### slices

In [34]:
mag_slice=magdata_rec[...,mask_edge:]
t=mag_slice.plot_quiver3d(mode='arrow', ar_dens=3)

In [35]:
# mag_slice.field = np.sqrt(np.abs(mag_slice.field)) * np.sign(mag_slice.field)
# too_big=mag_slice.field_amp>1.25
mag_slice=mag_slice[:,:,5:-10,5:]
mag_slice.field[mag_slice.field>1] = 1
t=mag_slice.plot_quiver3d(mode='arrow', ar_dens=3, labels=False)

In [36]:
slice_position=5
xoffset=30
yoffset=10
# navigator
mag_slice=pr.VectorData(1, magdata_rec.field[...,mask_edge:].copy())
mask = mag_slice.get_mask()
scalar = np.where(mask, 1, 0)
nav_field = pr.VectorData(1, np.array((scalar, np.zeros(mask.shape), np.zeros(mask.shape))))
nav_field.field[0,5,40,50]=4

# coloring nav
scalar[slice_position-1:slice_position+1,:,:]=scalar[slice_position-1:slice_position+1,:,:]*(-1)
#nav_field.field[:,slice_position,:,:] = nav_field.field[:,slice_position,:,:]*2
t=nav_field.plot_quiver3d(direction=(-20,-70,170) ,mode='cube', 
                          opacity=1, ar_dens=1, coloring='custom', custom_scalar=scalar, line_width=1,
                          labels=False, grid=False, orientation=False) #'arrow'

slice_mask = np.full(mask.shape, True)
slice_mask=np.array([slice_mask]*3)
slice_mask[:,slice_position:slice_position+1,:,:]=False

mag_slice.field = np.sqrt(np.abs(mag_slice.field)) * np.sign(mag_slice.field)
mag_slice.field[slice_mask]=0
mag_slice=mag_slice[:,:,:-yoffset,xoffset:]
t=mag_slice.plot_quiver3d(direction=(90,0,60), mode='arrow', ar_dens=1, coloring='angle', 
                          labels=False, orientation=False, grid=False)


In [38]:
magdata_rec.plot_quiver3d()
save_images=True
file=r"./gifs/slices"
for i in range (3,8)[::2]:

    slice_position=i
    xoffset=30
    yoffset=10
    # navigator
    mag_slice=pr.VectorData(1, magdata_rec.field[...,mask_edge:].copy())
    mask = mag_slice.get_mask()
    scalar = np.where(mask, 1, 0)
    nav_field = pr.VectorData(1, np.array((scalar, np.zeros(mask.shape), np.zeros(mask.shape))))
    nav_field.field[0,5,40,50]=4


    scalar[slice_position:slice_position+2,:,:]=scalar[slice_position:slice_position+2,:,:]*(-1)
    #nav_field.field[:,slice_position,:,:] = nav_field.field[:,slice_position,:,:]*2
    tn=nav_field.plot_quiver3d(direction=(-20,-70,200) ,mode='cube', 
                              opacity=1, ar_dens=1, coloring='custom', custom_scalar=scalar, line_width=1,
                              labels=False, grid=False, orientation=False) #'arrow'

    slice_mask = np.full(mask.shape, True)
    slice_mask=np.array([slice_mask]*3)
    slice_mask[:,slice_position,:,:]=False

    mag_slice.field = np.sqrt(np.abs(mag_slice.field)) * np.sign(mag_slice.field)
    mag_slice.field[slice_mask]=0
    mag_slice=mag_slice[:,:,:-yoffset,xoffset:]
    t=mag_slice.plot_quiver3d(direction=(90,0,60), mode='arrow', ar_dens=1, coloring='angle', 
                              labels=False, orientation=False, grid=False)
    if save_images:
        img = Image.fromarray(tn[1])
        img=img.convert("RGBA")
        fp_out=file+str(i)+"_nav.png"
        img.save(fp=fp_out, format='png', interlace=False)
        
        img = Image.fromarray(t[1])
        img=img.convert("RGBA")
        fp_out=file+str(i)+"_slice.png"
        img.save(fp=fp_out, format='png', interlace=False)

In [ ]:
mag_slice.shape

In [ ]:
mag_slice=magdata_rec[:,1:,1:,mask_edge:]
mean_field=np.mean(mag_slice.field, axis=1)
mean_field=np.pad(mean_field, ((0,0),(10,0,),(1,1)))
mag_field=np.zeros((3,1,*mean_field.shape[-2:]))
mag_field[:,0,:,:]=mean_field
mag_slice=pr.VectorData(mag_slice.a, mag_field)
mag_slice.field = np.sqrt(np.abs(mag_slice.field)) * np.sign(mag_slice.field)
print(mag_slice.field.shape)
mag_slice.plot_quiver_field(ar_dens=2, scale=0.5)

In [ ]:
np.pad?

In [ ]:
print(slice_mask.shape, mag_slice.field.shape)

In [ ]:
angle_field = pre.inspect_magdata(magdata_rec[:,:,:,mask_edge:], plot_angles=True)

In [15]:
#normalise field:
angles = angle_field.field.copy()
vectors=magdata_rec[...,mask_edge:]
angles[angles>90]=90
mask = vectors.get_mask()
for i in range(3):
    vectors.field[i,mask]=vectors.field[i,mask]/vectors.field_amp[mask]*(angles[mask])
t=vectors.plot_quiver3d(mode='sphere', opacity=0.3, ar_dens=1, coloring='custom', custom_scalar=angles) #'arrow'

NameError: name 'angle_field' is not defined

In [20]:
'max spin plot'

f = np.max(angle_field.field,axis=0)
f[f>90]=91
f[f<0]=0
f_shaped=np.ones((1,*f.shape))
f_shaped[0,:,:]=f
f_shaped = np.pad(f_shaped, ((0,0),(20,0),(0,1)))
print(f_shaped.shape)
maxang = pr.ScalarData(10, f_shaped)
maxang.plot_field()

(1, 113, 65)


In [ ]:
data_combined.plot_phasemaps()

In [ ]:
data_combined.plot_mask(labels=False, pretty=True)

### plots for projection maps

In [11]:
data=data_combined
masks=[]
titles=[]
folder=".//gifs"
for i in range(data.count):
    pm=data.phasemaps[i]
    pr=data.projectors[i]
    masks.append(pm.mask)
    xa=round(np.degrees(pr.tilt))
    za=round(np.degrees(pr.rotation))
    title=f"{za}.{xa}.png"
    titles.append(title)
pu.matshow_n(masks, titles)

# for i in range(data.count):
    

In [13]:
titles

['7.1.png',
 '7.21.png',
 '7.31.png',
 '7.-9.png',
 '7.-19.png',
 '7.-29.png',
 '7.-39.png',
 '7.-49.png',
 '7.-59.png',
 '-90.-4.png',
 '-90.-14.png',
 '-90.-24.png',
 '-90.-34.png',
 '-90.-44.png',
 '-90.-54.png',
 '-90.-62.png']